In [38]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

import requests 
from bs4 import BeautifulSoup

url = 'https://yandex.by/search/'
params={
        'text':'Xiaomi 14',
        'lr':157,
        'search_source':'yaby_desktop_common',
        'src':'suggest_B',
    }
for i, (k,v in) enumerate(params.items()):
    if i == 0:
        prefix = '?'
    else:
        prefix = '&'
      url += prefix + str(k) + '=' + str(v).replace(' ', '+')

from playwright.sync_api import sync_playwright
playwright = sync_playwright().start()
# Use playwright.chromium, playwright.firefox or playwright.webkit
# Pass headless=False to launch() to see the browser UI
browser = playwright.firefox.launch(headless=True)
page = browser.new_page()
page.goto(url)
page.screenshot(path="example2.png")
browser.close()
playwright.stop()


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [34]:
url += '&' + str(k).replace(' ', '+') + '=' + str(v) 

In [39]:
url


'https://yandex.by/search/&text=Xiaomi+14&lr=157&search_source=yaby_desktop_common&src=suggest_B'

NameError: name 'page' is not defined

In [19]:
ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

In [20]:
response = requests.get(
    url='https://yandex.by/search/',
    params={
        'text':'Xiaomi 14',
        'lr':157,
        'search_source':'yaby_desktop_common',
        'src':'suggest_B',
    },
    headers={'User-Agent': ua}
)

In [21]:
response

<Response [200]>

In [22]:
soup = BeautifulSoup(response.text, "html.parser")

In [23]:
soup.contents

['html',
 <html prefix="og: http://ogp.me/ns#"><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/>
 <link href="https://yastatic.net/s3/home-static/_/a6/a6a296b741b51880ae8a9b04a67cfe3f.png" rel="icon" sizes="16x16"/>
 <link href="https://yastatic.net/s3/home-static/_/f4/f47b1b3d8194c36ce660324ab55a04fe.png" rel="icon" sizes="32x32"/>
 <link href="https://yastatic.net/s3/home-static/_/f0/f0597b6727cc67dceebc4e3a87caf571.png" rel="icon" sizes="192x192"/>
 <link href="https://yastatic.net/s3/home-static/_/a7/a79b81aa025e9edb2244e38581c868ad.png" rel="apple-touch-icon" sizes="152x152"/>
 <link href="https://yastatic.net/s3/home-static/_/46/462e92b9e3792be37a1c3fdefb26af28.png" rel="apple-touch-icon" sizes="180x180"/>
 <title data-react-helmet="true">ÐÐ¹, ÐÐ°Ð¿ÑÐ°!</title><meta content="Ð¯Ð½Ð´ÐµÐºÑ" data-react-helmet="true" property="og:title"/><meta content="ÐÐ°Ð¹Ð´ÑÑÑÑ Ð²ÑÑ" data-reac

In [17]:
from selenium import webdriver

driver = webdriver.Chrome()  # can be webdriver.Chrome()
driver.get("about:blank")

data = '<h1>test</h1>'  # supposed to come from BeautifulSoup
driver.execute_script('document.body.innerHTML = "{html}";'.format(html=soup))

JavascriptException: Message: javascript error: Invalid or unexpected token
  (Session info: chrome=120.0.6099.71)
Stacktrace:
#0 0x564f9f274f83 <unknown>
#1 0x564f9ef2dcf7 <unknown>
#2 0x564f9ef342a3 <unknown>
#3 0x564f9ef36bb4 <unknown>
#4 0x564f9efc6ba3 <unknown>
#5 0x564f9efa70b2 <unknown>
#6 0x564f9efc6006 <unknown>
#7 0x564f9efa6e53 <unknown>
#8 0x564f9ef6edd4 <unknown>
#9 0x564f9ef701de <unknown>
#10 0x564f9f239531 <unknown>
#11 0x564f9f23d455 <unknown>
#12 0x564f9f225f55 <unknown>
#13 0x564f9f23e0ef <unknown>
#14 0x564f9f20999f <unknown>
#15 0x564f9f262008 <unknown>
#16 0x564f9f2621d7 <unknown>
#17 0x564f9f274124 <unknown>
#18 0x787605494ac3 <unknown>


In [22]:
!pip install selenium

  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 10.8 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 9.4 MB/s eta 0:00:006 MB/s eta 0:00:01
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [20]:
type(soup)

bs4.BeautifulSoup

In [ ]:
start=0
  &num=10
  &q=red+sox
  &cr=countryCA
  &lr=lang_fr
  &client=google-csbe
  &output=xml_no_dtd
  &cx=00255077836266642015:u-scht7a-8i

In [10]:
with open('google_res.html', 'w') as f:
    f.write(response.text)
# response.text

In [13]:


headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }


def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get)

    soup = BeautifulSoup(r.text, "html.parser")
    output = []
    for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
        url = searchWrapper.find('a')["href"] 
        text = searchWrapper.find('a').text.strip()
        result = {'text': text, 'url': url}
        output.append(result)

    return output

In [14]:
output = google('Xiaomi 14')

In [15]:
output

[]

In [35]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass `google_api_key` as a named parameter. (type=value_error)

In [25]:
import requests

In [26]:
user_query = 'яндекс маркет отзывы Xiaomi 14'
query = f'https://www.google.com/search?q={user_query}'

In [27]:
results = requests.get(query)

In [20]:
search = DuckDuckGoSearchAPIWrapper(region="ru-ru", time="d", max_results=20, backend='html')
# searching_tool = DuckDuckGoSearchResults(api_wrapper=search,)

In [21]:
result = searching_tool.invoke('яндекс маркет отзывы')

In [22]:
search_run = DuckDuckGoSearchRun()

In [11]:
result = search_run.invoke('яндекс маркет отзывы')

In [14]:
search.invoke('яндекс маркет отзывы')

AttributeError: 'DuckDuckGoSearchAPIWrapper' object has no attribute 'invoke'

In [23]:
search.run(query='яндекс маркет отзывы Xiaomi 14')

'В DNS — 65 999 ₽ на «Яндекс Маркете» — от 50 959 ₽ ... тесты коллег-журналистов и отзывы пользователей, мы отобрали лучшие ноутбуки в бюджете до 70 000 ₽. ... Xiaomi Redmi Book Pro 14. Asus VivoBook 17 X1704ZA-AU121W. Изучаете отзывы про ТОП—7. Лучшие ноутбуки 13-14 дюймов. Рейтинг 2023 года!? Почитайте свежие реальные отзывы от имени бывших сотрудников на сайте shitcompany.org Стоит ли покупать Xiaomi Poco F5 Pro зимой 2023-2024 года? / Арстайл / - тема важная и интересная, поэтому наша редакция подготовила подробный разбор этой темы на сайте mnogorabotnikov.ru Лучшие смартфоны без ШИМ: Топ-5 смартфонов с IPS-экраном 📱 Рейтинг 2024 года - тема важная и интересная, поэтому наша редакция подготовила подробный разбор этой темы на сайте mnogorabotnikov.ru Изучаете отзывы про ТОП-6. Лучшие ноутбуки для работы и учебы ⚡ Рейтинг 2024 года по цене-качеству? Почитайте свежие реальные отзывы от имени бывших сотрудников на сайте shitcompany.org От 5 до 10 тысяч рублей выручают 16% респондентов,